<a href="https://colab.research.google.com/github/Nuha4/adelaide_metrocard-cpi-fuel-correlation/blob/main/adl_metrocard_cpi_fuel_correlation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr, spearmanr
import numpy as np
import glob
import os

In [ ]:
# Load, Merge and Clean MetroCard Usage Data for Adelaide (2015_Q4 to 2025_Q2)
folder_path = '.'
file_pattern = os.path.join(folder_path, "*anded*.csv")
all_files = glob.glob(file_pattern)

# Read and concatenate
df_list = []
for file in all_files:
    df = pd.read_csv(file)
    df['SOURCE_FILE'] = os.path.basename(file)  # Optional: track source file
    df_list.append(df)
combined_metro_df = pd.concat(df_list, ignore_index=True)

# Convert date and prepare monthly data
combined_metro_df['VALIDATION_DATE'] = pd.to_datetime(combined_metro_df['VALIDATION_DATE'], dayfirst=True)
combined_metro_df['BAND_BOARDINGS_FLOOR'] = pd.to_numeric(combined_metro_df['BAND_BOARDINGS_FLOOR'], errors='coerce')
combined_metro_df['Month'] = combined_metro_df['VALIDATION_DATE'].dt.to_period('M').dt.to_timestamp()

# Aggregate by month
monthly_metro = combined_metro_df.groupby('Month')['BAND_BOARDINGS_FLOOR'].sum().reset_index()
monthly_metro = monthly_metro.rename(columns={'BAND_BOARDINGS_FLOOR': 'MetroCard_Usage'})

# Save for reuse
monthly_metro.to_csv("combined_metrocard_monthly.csv", index=False)
print(monthly_metro.head())
